In [1]:
# Import neccesary programs
import requests
import bs4
from bs4 import BeautifulSoup
from lxml import html
import numpy as np
import pandas as pd

### Webscraping Functions

In [2]:
# function for getting location from indeed

def extract_location_from_result(loc):
    try:
        for n in loc.findAll(class_='location'):
            return n.text.strip()
    except:
        return 'None'


In [3]:
# function for getting company from indeed

def extract_company_from_result(comp):
    try:
        for n in comp.findAll(class_='company'):
            return n.text.strip()
    except:
        return 'None' 

In [4]:
# function for getting job title from indeed

def extract_jobtitle_from_result(jobtitle):
    try:
        for n in jobtitle.findAll(class_='jobtitle'):
            return n.text.strip()
    except:
        return 'None'   

In [5]:
# function to get salary from indeed

def extract_salary_from_result(sal):
    try:
        for n in sal.findAll('td', class_='snip'):
            return n.nobr
    except:
        return 'None' 

In [6]:
# function to get summary/description from indeed

def extract_summary_from_result(desc):
    try:
        for n in desc.findAll('span', class_='summary'):
            return n.text.strip()
    except:
        return 'None' 

### Select Website 

In [7]:
Location = []
Job_Title = []
Company = []
Salary = []
Description = []

url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100
cities = ['New+York', 'Chicago', 'Boston', 'San+Francisco', 'Los+Angeles', 'Austin', 'Atlanta']
results = []
for city in set(cities):
    for start in range(0, max_results_per_city, 10):
        url = url_template.format(city,start)
        ru = requests.get(url)
        indeed = BeautifulSoup(ru.content, "lxml")
        for item in indeed.findAll('div', class_='result'):
            Location.append(extract_location_from_result(item))
            Job_Title.append(extract_jobtitle_from_result(item))
            Company.append(extract_company_from_result(item))
            Salary.append(extract_salary_from_result(item))
            Description.append(extract_summary_from_result(item))

In [ ]:
city_pages = []
for city in set(cities):
    for i in range(int(max_results_per_city)/10):
        city_pages.append(city)
cities = pd.DataFrame({'city':city_pages})
city_pages.append(city)


In [8]:
df = pd.DataFrame([Location,Job_Title,Company,Salary,Description]).T 
  
df.columns= ['Location','Job_Title','Company','Salary','Description']

In [ ]:
df.head()

In [ ]:
df.dtypes

### Cleaning Salary Data

In [9]:
df['Salary'].unique()

array([None, <nobr>$4,584 a month</nobr>,
       <nobr>$6,250 - $10,833 a month</nobr>,
       <nobr>$4,599 - $6,066 a month</nobr>, <nobr>$6,667 a month</nobr>,
       <nobr>$5,541 a month</nobr>, <nobr>$5,400 - $6,500 a month</nobr>,
       <nobr>$4,000 a month</nobr>, <nobr>$3,520 - $4,600 a month</nobr>,
       <nobr>$4,916 a month</nobr>, <nobr>$5,259 - $8,624 a month</nobr>,
       <nobr>$140,000 - $160,000 a year</nobr>,
       <nobr>$23.44 - $29.80 an hour</nobr>,
       <nobr>$5,259 - $6,941 a month</nobr>,
       <nobr>$4,500 - $4,900 a month</nobr>,
       <nobr>$120,000 - $150,000 a year</nobr>,
       <nobr>$120,000 a year</nobr>,
       <nobr>$80,000 - $110,000 a year</nobr>,
       <nobr>$180,000 a year</nobr>, <nobr>$110,000 a year</nobr>,
       <nobr>$104,674 - $151,358 a year</nobr>,
       <nobr>$105,000 a year</nobr>,
       <nobr>$135,000 - $165,000 a year</nobr>,
       <nobr>$110,000 - $155,000 a year</nobr>,
       <nobr>$100,000 - $150,000 a year</nobr>,
     

In [10]:
df = df[df.Salary.notnull()]

In [11]:
df['Salary'] = df['Salary'].apply(lambda x: str(x))
df['Salary'] = df['Salary'].apply(lambda x: x.replace('$',''))
df['Salary'] = df['Salary'].apply(lambda x: x.replace(',',''))
df['Salary'] = df['Salary'].apply(lambda x: x.replace('<nobr>',''))
df['Salary'] = df['Salary'].apply(lambda x: x.replace('</nobr>',''))



In [12]:
df['hourly_sal'] = df['Salary'].map(lambda x: 1 if 'hour' in x else 0)

df['monthly_sal'] = df['Salary'].map(lambda x: 1 if 'month' in x else 0)

df['yearly_sal'] = df['Salary'].map(lambda x: 1 if 'year' in x else 0)



In [13]:
df.head()

,Location,Job_Title,Company,Salary,Description,hourly_sal,monthly_sal,yearly_sal
3,"Austin, TX",Research Scientist,University of Texas at Austin,4584 a month,Provide statistical support on research and su...,0,1,0
8,"Austin, TX 78717",Research Scientist,The University of Texas at Austin,4584 a month,Provide statistical support on research and su...,0,1,0
18,"Austin, TX",Engineering Scientist - GPS Data Analyst,University of Texas at Austin,6250 - 10833 a month,Detailed data analyses of GPS observation and ...,0,1,0
26,"Austin, TX",Statistical Analyst,Health & Human Services Comm,4599 - 6066 a month,Creation of data appropriate statistical sampl...,0,1,0
52,"Austin, TX",Senior Research Scientist,University of Texas at Austin,6667 a month,Play leadership role in development of data co...,0,1,0
